In [ ]:

def GCTriINT(p, v1, v2, eta):
    # Calculate integral of green function (solution of lapalce equation)
    # Input
    # p, v1, v2 : 3d vector
    alpha = np.arccos(((v2-v1)@(p-v1))/(np.linalg.norm(v2-v1)*np.linalg.norm(p-v1)))
    beta = np.arccos(((v1-p)@(v2-p))/(np.linalg.norm(v1-p)*np.linalg.norm(v2-p)))
    
    lamda = (np.linalg.norm(p-v1)**2)*(np.sin(alpha)**2)
    c = np.linalg.norm(p-eta)**2
    
    theta = np.pi - alpha
    S = np.sin(theta)
    C = np.cos(theta)
    I1 = -np.sign(S)*(1/2)*[2*np.sqrt(c)*np.arctan(np.sqrt(c)*C/np.sqrt(lamda+c*S**2))+np.sqrt(lamda)*np.log(2*np.sqrt(lamda)*(S**2)*(1/(1-C)**2)*(1-2*c*C/(c*(1+C)+lamda+np.sqrt(lamda**2+lamda*c*S**2))))]
    
                            
    theta = np.pi - alpha - beta
    S = np.sin(theta)
    C = np.cos(theta)
    I2 = -np.sign(S)*(1/2)*[2*np.sqrt(c)*np.arctan(np.sqrt(c)*C/np.sqrt(lamda+c*S**2))+np.sqrt(lamda)*np.log(2*np.sqrt(lamda)*(S**2)*(1/(1-C)**2)*(1-2*c*C/(c*(1+C)+lamda+np.sqrt(lamda**2+lamda*c*S**2))))]
    
    return -(1/(4*np.pi))*abs(I1-I2-np.sqrt(c)*beta)
    
    
# η =Xi∈IV(φi(η)vi) + Xj∈IT(ψj(η)n(tj))
def gc(V, T, n, vi):
    # Input
    # V : (cv, 3) (cv: # of cage vertex) - cage vertices (sorted by ID)
    # T : (cf, 3) (cf: # of cage face) - vertices IDs of faces
    # n : (cf, 3) (cf: # of cage face) - normal vector of faces
    # vi : (n, 3) (n: # of shape vertex) - vertices of shape
    
    # Output
    # phi : (n, cv) : coordinate about cage's vertex 
    # psi : (n, cf) : coordinate about cage's face's normal
    
    phi = np.zeros((len(vi), len(V))) #len(a) : # of a's row
    psi = np.zeros((len(vi), len(T)))
    
    for vidx, eta in enumerate(vi):
        for fidx, f in enumerate(T):
            v = np.array([V[f[0]], V[f[1]], V[f[2]], V[f[0]]])
            v = v-eta
            p = (v[0]@n[fidx])*n[fidx]
            
            s =  np.zeros(3)
            I =  np.zeros(3)
            II = np.zeros(3)
            N = np.zeros((3, 3))
            
            for l in range(3):
                s[l] = np.sign(np.cross(v[l]-p, v[l+1]-p)@n[fidx])
                I[l] = np.GCTriINT(p, v[l], v[l+1], np.zeros(3))
                II[l] = np.GCTriINT(np.zeros(3), v[l], v[l+1], np.zeros(3))
                q = np.cross(v[l+1], v[l])
                N[l] = q/np.linalg.norm(q)
                if(np.linalg.norm(q[l]) < 0.00001): II[l] = 0
            
            I_ = abs(s[0]*I[0]+s[1]*I[1]+s[2]*I[2])
            psi[vidx, fidx] = I_
            w = n[fidx]*I_+N[0]*II[0]+N[1]*II[1]+N[2]*II[2]
            
            if np.linalg.norm(w) > 0.00001:
                for l in range(3):
                    phi[vidx, f[l]] += (N[(l+1)%3]@w)/(N[(l+1)%3]@v[l])
    
    return phi.T, psi.T

phi, phi = gc(v_c, f_c, v_mci)
    